In [68]:
import numpy as np
import pandas as pd
import pdfplumber

In [69]:
from collections import Counter
import math

In [70]:
def ekstrak_teks_dari_pdf(path):
    with pdfplumber.open(path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text(x_tolerance=2, y_tolerance=2)
    return text

In [71]:
# File yang sama
teks_1 = ekstrak_teks_dari_pdf("./jurnal/paper1.pdf")
print(teks_1[100:200])

lsevier.com/locate/displa
A unified architecture for super-resolution and segmentation of remote
✩
s


In [72]:
# Artikel rekomendasi dari science direct
teks_2 = ekstrak_teks_dari_pdf("./jurnal/paper2.pdf")
print(teks_2[100:200])

lsevier.com/locate/displa
✩
FSNet: A dual-domain network for few-shot image classification
∗
Xuewen 


In [73]:
# Artikel yang disitasi paper ini
teks_3 = ekstrak_teks_dari_pdf("./jurnal/paper3.pdf")
print(teks_3[:100])

Multi-Scale High-Resolution Vision Transformer for Semantic Segmentation
Jiaqi Gu2*, Hyoukjun Kwon1,


In [74]:
# Artikel dengan author yang sama (jurnal beda)
teks_4 = ekstrak_teks_dari_pdf("./jurnal/paper4.pdf")
print(teks_4[100:200])

e at ScienceDirect
Engineering Applications of Artificial Intelligence
journal homepage: www.elsevie


In [75]:
# Artikel yang menyitasi paper ini
teks_5 = ekstrak_teks_dari_pdf("./jurnal/paper5.pdf")
print(teks_5[100:200])

mepage: www.elsevier.com/locate/neucom
A multimodal fusion framework for semantic segmentation of re


In [76]:
def tokenize(text):
    return text.lower().split()

In [77]:
def ngrams(tokens, n):
    return [tuple(tokens[i:i+n]) for i in range(len(tokens)-n+1)]

In [78]:
def cosine_similarity(ngrams1, ngrams2):
    c1, c2 = Counter(ngrams1), Counter(ngrams2)
    all_keys = set(c1.keys()) | set(c2.keys())
    
    dot = sum(c1[k] * c2[k] for k in all_keys)
    norm1 = math.sqrt(sum(v**2 for v in c1.values()))
    norm2 = math.sqrt(sum(v**2 for v in c2.values()))
    
    return dot / (norm1 * norm2) if norm1 and norm2 else 0

In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity as cosine_sim_sklearn

def cek_n_gram_all(text_1, text_2):
    tokens1 = tokenize(text_1)
    tokens2 = tokenize(text_2)

    hasil = {}
    for n in [1, 2, 3]:
        # Cosine dengan n-gram manual
        ngrams1 = ngrams(tokens1, n)
        ngrams2 = ngrams(tokens2, n)
        cosine = cosine_similarity(ngrams1, ngrams2)
        hasil[f"Cosine_n{n}"] = cosine

        # Jaccard
        set1, set2 = set(ngrams1), set(ngrams2)
        jaccard = len(set1 & set2) / len(set1 | set2) if set1 | set2 else 0
        hasil[f"Jaccard_n{n}"] = jaccard

        # TF-IDF + Cosine
        vectorizer = TfidfVectorizer(ngram_range=(n, n))
        tfidf = vectorizer.fit_transform([text_1, text_2])
        tfidf_cosine = cosine_sim_sklearn(tfidf[0:1], tfidf[1:2])[0][0]
        hasil[f"TFIDF_Cosine_n{n}"] = tfidf_cosine

    return hasil




In [95]:
perbandingan = [
    ("Dua file yang sama", teks_1, teks_1),
    ("Artikel yang direkomendasikan oleh ScienceDirect", teks_1, teks_2),
    ("Artikel yang disitasi paper ini", teks_1, teks_3),
    ("Artikel dengan author yang sama", teks_1, teks_4),
    ("Artikel yang menyitasi paper ini", teks_1, teks_5),
]

hasil = []
for n in [1, 2, 3]:
    for label, t1, t2 in perbandingan:
        metrics = cek_n_gram_all(t1, t2)
        hasil.append({
            "n": n,
            "Perbandingan": label,
            "Cosine": metrics[f"Cosine_n{n}"],
            "Jaccard": metrics[f"Jaccard_n{n}"],
            "TFIDF_Cosine": metrics[f"TFIDF_Cosine_n{n}"]
        })

df = pd.DataFrame(hasil)

# Pastikan urut sesuai n lalu perbandingan
df = df.sort_values(by=["n"]).reset_index(drop=True)

print(df)


    n                                      Perbandingan    Cosine   Jaccard  \
0   1                                Dua file yang sama  1.000000  1.000000   
1   1  Artikel yang direkomendasikan oleh ScienceDirect  0.900487  0.201202   
2   1                   Artikel yang disitasi paper ini  0.800596  0.156098   
3   1                   Artikel dengan author yang sama  0.887178  0.237533   
4   1                  Artikel yang menyitasi paper ini  0.901507  0.215031   
5   2                                Dua file yang sama  1.000000  1.000000   
6   2  Artikel yang direkomendasikan oleh ScienceDirect  0.421375  0.052656   
7   2                   Artikel yang disitasi paper ini  0.215511  0.032057   
8   2                   Artikel dengan author yang sama  0.398877  0.081555   
9   2                  Artikel yang menyitasi paper ini  0.449717  0.063186   
10  3                                Dua file yang sama  1.000000  1.000000   
11  3  Artikel yang direkomendasikan oleh ScienceDir

In [96]:
df.head()

,n,Perbandingan,Cosine,Jaccard,TFIDF_Cosine
0,1,Dua file yang sama,1.000000,1.000000,1.000000
1,1,Artikel yang direkomendasikan oleh ScienceDirect,0.900487,0.201202,0.852079
2,1,Artikel yang disitasi paper ini,0.800596,0.156098,0.747718
3,1,Artikel dengan author yang sama,0.887178,0.237533,0.891466
4,1,Artikel yang menyitasi paper ini,0.901507,0.215031,0.871168


In [97]:
latex_table = df.to_latex(index=False, float_format="%.4f")

print(latex_table)

\begin{tabular}{rlrrr}
\toprule
n & Perbandingan & Cosine & Jaccard & TFIDF_Cosine \\
\midrule
1 & Dua file yang sama & 1.0000 & 1.0000 & 1.0000 \\
1 & Artikel yang direkomendasikan oleh ScienceDirect & 0.9005 & 0.2012 & 0.8521 \\
1 & Artikel yang disitasi paper ini & 0.8006 & 0.1561 & 0.7477 \\
1 & Artikel dengan author yang sama & 0.8872 & 0.2375 & 0.8915 \\
1 & Artikel yang menyitasi paper ini & 0.9015 & 0.2150 & 0.8712 \\
2 & Dua file yang sama & 1.0000 & 1.0000 & 1.0000 \\
2 & Artikel yang direkomendasikan oleh ScienceDirect & 0.4214 & 0.0527 & 0.2611 \\
2 & Artikel yang disitasi paper ini & 0.2155 & 0.0321 & 0.1540 \\
2 & Artikel dengan author yang sama & 0.3989 & 0.0816 & 0.3724 \\
2 & Artikel yang menyitasi paper ini & 0.4497 & 0.0632 & 0.3494 \\
3 & Dua file yang sama & 1.0000 & 1.0000 & 1.0000 \\
3 & Artikel yang direkomendasikan oleh ScienceDirect & 0.0741 & 0.0141 & 0.0452 \\
3 & Artikel yang disitasi paper ini & 0.0200 & 0.0055 & 0.0171 \\
3 & Artikel dengan author yang sa